We can start by importing our favourite package: ``elk``!

In [21]:
import elk
import numpy as np

%config InlineBackend.figure_format = "retina"

## Lightcurve Setup

Now let's do a simple ensemble lightcurve fit to NGC 419

In [22]:
c = elk.ensemble.EnsembleLC(output_path=None,
                            cluster_name='NGC 419',
                            location='23.58271, +61.1236',
                            radius=.046,
                            cluster_age=7.75,
                            cutout_size=10,
                            debug=True,
                            verbose=True,
                            no_lk_cache=False)

In [23]:
c.create_output_table()

NGC 419 has 4 observations
Starting Quality Tests for Observation: 0
Failed Near Edge Test
Starting Quality Tests for Observation: 1


100%|██████████| 100/100 [00:02<00:00, 44.64it/s]


1 Passed Quality Tests
Starting Quality Tests for Observation: 2


100%|██████████| 100/100 [00:02<00:00, 33.94it/s]


2 Passed Quality Tests
Starting Quality Tests for Observation: 3
Failed Near Edge Test


name,location,radius,log_age,has_data,n_obs,n_good_obs,n_failed_download,n_near_edge,n_scatter_light,lc_lens,which_sectors_good
str7,str18,float64,float64,bool,int64,int64,int64,int64,int64,int64[2],int64[2]
NGC 419,"23.58271, +61.1236",0.046,7.75,True,4,2,0,2,0,1196 .. 3351,2 .. 27


## Accessing statistics

Now that we've run that we can use the output to start analysing some statistics about the lightcurves that we've created. Let's pick one of them out.

In [24]:
lc = c.lcs[2]

This lightcurve has access to the many functions within ``elk.stats`` and we can easily access them. For the statistics that don't require inputs, you can simply access them as attributes.

In [25]:
lc.rms, lc.von_neumann_ratio

(0.9999983, 0.00017164927292955188)

Similarly, for statistics that could get some input you can access them as class methods like this

In [26]:
lc.J_stetson()

0.6518914852746158

Some of the statistics also _require_ input, like when you create a periodogram

In [27]:
frequencies = np.logspace(-1, 1, 100)
_ = lc.to_periodogram(frequencies=frequencies, n_bootstrap=10)
lc.stats["n_peaks"]

2

## Summarising Statistics

As you may have noticed in the last cell, each of the statistics has been tracks in a dictionary within the class to save you running them again. You can access this dictionary like this

In [28]:
lc.stats

{'rms': 0.9999983,
 'von_neumann_ratio': 0.00017164927292955188,
 'J_Stetson': 0.6518914852746158,
 'max_power': 0.007725125075032853,
 'freq_at_max_power': 1.2328467394420666,
 'peak_freqs': array([0.97700996, 1.23284674, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 'power_at_peaks': array([0.00670849, 0.00575052, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 'n_peaks': 2,
 'ratio_of_power_at_high_v_low_freq': 2.614286024689913}

You can also turn this into an Astropy table for aggregation with other lightcurves

In [29]:
lc.get_stats_table(name=c.cluster_name)

name,rms,von_neumann_ratio,J_Stetson,max_power,freq_at_max_power,peak_freqs,power_at_peaks,n_peaks,ratio_of_power_at_high_v_low_freq
str7,float32,float64,float64,float64,float64,float64[25],float64[25],int64,float64
NGC 419,0.9999983,0.00017164927292955188,0.6518914852746158,0.007725125075032853,1.2328467394420666,0.9770099572992255 .. 0.0,0.006708490384895873 .. 0.0,2,2.614286024689913


## The Kitchen Sink

If you're in an instance in which you want to get every statistic for the lightcurve quickly, you can run them all **using default settings** as follows

In [34]:
lc.get_stats_using_defaults()

{'rms': 0.03631488075285084,
 'von_neumann_ratio': 0.00017164927292955188,
 'J_Stetson': 0.6518914852746158,
 'max_power': 0.01619743375840398,
 'freq_at_max_power': 0.9600000000000002,
 'peak_freqs': array([0.96, 1.1 , 1.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  ]),
 'power_at_peaks': array([0.01268922, 0.00297482, 0.01128423, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 'n_peaks': 3,
 'ratio_of_power_at_high_v_low_freq': 1.9553476030324772,
 'std': 0.00015886688,
 'MAD': 0.000105917454,
 'max_autocorrelation': 0.15670049925982307,
 'time_of_max_autocorrelation': array([0.06944564]),
 'skewness': -52.04516}

We recommend caution when using this function since the default settings may not be the best ones for you!

Now when converting back to a table we can see _every_ statistic

In [35]:
lc.get_stats_table(name=c.cluster_name)

name,rms,von_neumann_ratio,J_Stetson,max_power,freq_at_max_power,peak_freqs,power_at_peaks,n_peaks,ratio_of_power_at_high_v_low_freq,std,MAD,max_autocorrelation,time_of_max_autocorrelation,skewness
str7,float64,float64,float64,float64,float64,float64[25],float64[25],int64,float64,float32,float32,float64,float64[1],float32
NGC 419,0.03631488075285084,0.00017164927292955188,0.6518914852746158,0.01619743375840398,0.9600000000000002,0.9600000000000002 .. 0.0,0.012689219386560755 .. 0.0,3,1.9553476030324772,0.00015886688,0.000105917454,0.15670049925982307,0.0694456417299989,-52.04516
